In [85]:
# 現状の大切な人たちとの時間を集計し、彼らと過ごすべき理想的な時間を特定する
# House、Friendsの時間が大切な人たちのとの時間に該当する
# リクルートに入ってからのHouse、Friendsの時間が理想的な時間にあたる認識であるため、
# リクルートに入ってから取得している日時のHouse、Friendsのデータを平均した値を理想的な1日あたりの時間とする
# なお、リクルートに入ったのは2022/01、日時の取得開始日は2023/09/25、よって2023/09/25から最新データまでの平均した値を理想的な1日あたりの時間とする
# ただし、各分類の割合を算出した際に、"多い"と感じれば、一旦検討する。
# これを振り返りの頻度分、例えば30年分合計することで、30年間の大切な人たちとの過ごした時間の目標を設定する

In [11]:
import gspread
import numpy as np
import pandas as pd
from google.oauth2.service_account import Credentials

# 認証情報のファイルパス
SERVICE_ACCOUNT_FILE = "./time-checker-410603-30bc9b39c86e.json"

# 認証スコープ
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# 認証情報の設定
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# クライアントの作成
client = gspread.authorize(credentials)

# スプレッドシートのIDとシート名
SPREADSHEET_ID = "10m-t07dQNktoX_bmQEAmtqhd9wDDFFMIWtjo8bgAHMw"
SHEET_NAME = "raw_raw"

# スプレッドシートのデータを取得
sheet = client.open_by_key(SPREADSHEET_ID)
worksheet = sheet.worksheet(SHEET_NAME)
data = worksheet.get()

# # データの表示
# for row in data:
#     print(row)

In [77]:
arr = np.array(data)
df_h = pd.DataFrame(arr[:, 1:].transpose(), columns=arr[:, 0])
df_h_tmp = df_h.copy()

# 各カラムの型を指定
df_h_tmp["date"] = pd.to_datetime(df_h_tmp["date"])
df_h_tmp["weekday"] = df_h_tmp["weekday"].astype(str)

# 特定の列に`#NUM`が含まれるレコードを抽出
invalid_records = df_h_tmp[
    df_h_tmp.apply(lambda row: row.astype(str).str.contains("#NUM!").any(), axis=1)
]
# 各レコードにおいて、不正な値を含んでいるカラム数を計算
df_h_tmp["Invalid_Count"] = df_h_tmp.apply(
    lambda row: row.astype(str).str.contains("#NUM!").sum(), axis=1
)

# float_columns
float_columns = [
    "Sleep",
    "MorningRoutine",
    "Personal",
    "RegularRevenue",
    "House",
    "Vook&NewBusiness",
    "Friends",
    "Training",
    "Move",
    "Study",
    "sum",
]

# sumから他の列の値を引いて残りの時間を計算
df_h_tmp = df_h_tmp.replace("#NUM!", 0.0)

float_columns = [
    "Sleep",
    "MorningRoutine",
    "Personal",
    "RegularRevenue",
    "House",
    "Vook&NewBusiness",
    "Friends",
    "Training",
    "Move",
    "Study",
    "sum",
]
df_h_tmp[float_columns] = df_h_tmp[float_columns].astype(float)

# sumから他の列の値を引いて残りの時間を計算
df_h_tmp["Remaining_Time"] = df_h_tmp["sum"] - df_h_tmp[float_columns[:-1]].sum(axis=1)

# 残りの時間を不正な値の数で割ることで#NUMを補完
df_h_tmp["Replacement_Value"] = df_h_tmp["Remaining_Time"] / df_h_tmp["Invalid_Count"]

# 不正な値を補完
for col in float_columns[:-1]:
    for (_, row_org), (_, row_tmp) in zip(df_h.iterrows(), df_h_tmp.iterrows()):
        if row_org[col] == "#NUM!":
            row_org[col] = row_tmp["Replacement_Value"]

# 全てのfloat_columns列に"#NUM!"が含まれていないことを確認する
for col in float_columns:
    assert not df_h[col].str.contains("#NUM!").any(), f"Column {col} contains '#NUM!'"

df_v = pd.melt(
    df_h, id_vars=["date", "weekday"], var_name="category", value_name="time"
)
df_v["time"] = df_v["time"].astype(float)

# NULLチェック
assert not df_v.isnull().any().any()

# データ型チェック
assert df_v.dtypes.tolist() == [
    np.dtype("O"),
    np.dtype("O"),
    np.dtype("O"),
    np.dtype("float64"),
]

# データマート作成
datamart = df_v.copy()

In [122]:
output_tmp = datamart[["category", "time"]].groupby("category").mean().reset_index()
output_tmp["rate"] = (
    output_tmp["time"] / output_tmp[output_tmp["category"] == "sum"]["time"].iloc[0]
)
output_tmp = output_tmp.sort_values("rate", ascending=False)

# 各分類に費やす1日あたりの時間の割合の合計が2に一致するか確認 NOTE: sumも含むので。
assert output_tmp["rate"].sum() == 2.0, print(output_tmp["rate"].sum())
output = output_tmp[output_tmp["category"] != "sum"].reset_index(drop=True)

# 1日あたりのSleep（睡眠のための）の時間を出力する。
time_sleep = output[output["category"] == "Sleep"]["time"].iloc[0]


# 1日あたりのRegularRevenue + Vook&NewBusiness（事業のための）の時間を出力する。
time_labor = (
    output[output["category"] == "RegularRevenue"]["time"].iloc[0]
    + output[output["category"] == "Vook&NewBusiness"]["time"].iloc[0]
)


# 1日あたりのHouse + Friends（大切な人のための）の時間を出力する。
time_precious = (
    output[output["category"] == "House"]["time"].iloc[0]
    + output[output["category"] == "Friends"]["time"].iloc[0]
)


# 1日あたりのそれ以外の時間を出力する。
time_others = output[
    ~output["category"].isin(
        ["Sleep", "RegularRevenue", "Vook&NewBusiness", "House", "Friends"]
    )
]["time"].sum()

#
assert time_sleep + time_labor + time_precious + time_others == 24.0
print("睡眠のための時間/日:", round(time_sleep, 2))
print("事業のための時間/日:", round(time_labor, 2))
print("大切な人のための時間/日:", round(time_precious, 2))
print("それ以外の時間/日:", round(time_others, 2))

睡眠のための時間/日: 8.76
事業のための時間/日: 7.72
大切な人のための時間/日: 3.8
それ以外の時間/日: 3.72


In [124]:
year = 30
print(
    f"大切な人たちと多くの時間を過ごすという目的に対応する目標時間（{year}年）:",
    int(round(time_precious * 365 * year, 0)),
)

大切な人たちと多くの時間を過ごすという目的に対応する目標時間（30年）: 41662
